In [4]:
# Import packages
import numpy as np
import pandas as pd
import math
import datetime
import os
import matplotlib.pyplot as plt
import warnings
import collections
import statsmodels.api as sm
from scipy.stats import t
from sklearn.utils import check_array
from functools import reduce
import seaborn as sns
import warnings
import collections
from matplotlib.pylab import rcParams
warnings.simplefilter(action='ignore', category=Warning)
import seaborn as sns
sns.set()
pd.options.mode.chained_assignment = None  # default='warn'
sns.set()

# Diebold-Mariano test


To test the forecast performance of method (1) versus (2), the following test statistic is defined:


### Test Statistic: $D M_{12}=\frac{\bar{d}_{12}}{\widehat{\sigma}_{\bar{d}_{12}}}$



### where $d_{12, t+1}=\frac{1}{n_{3, t+1}} \sum_{i=1}^{n_{3, t+1}}\left(\left(\hat{e}_{i, t+1}^{(1)}\right)^{2}-\left(\hat{e}_{i, t+1}^{(2)}\right)^{2}\right)$


$\hat{e}_{i, t+1}^{(1)}$ and $\hat{e}_{i, t+1}^{(2)}$ denote the prediction error for stock return i at time t from column model 1 and row model $2,  n_{3, t+1}$ accounts for the number of stocks in the testing sample year $\mathrm{t}+1$ and $\bar{d}_{12}$ and $\hat{\sigma}_{\bar{d}_{12}}$ denote mean and Newey-West standard error of $d_{12, t+1}$ over the testing set.

In [5]:
def dm_test(e1, e2, alternative='two_sided', h=1, power=2):
    """
    The Diebold-Mariano test for predictive accuracy.
    The DM test compares the forecast accuracy of different forecasting methods. This implementation is designed
    according to R implementation which could be found here: https://pkg.robjhyndman.com/forecast/reference/dm.test.html
    :param e1: forecast errors from the first method
    :param e2: forecast errors from the second method
    :param alternative: str specifying the alternative hypothesis, 'two_sided' (default one), 'less' or 'greater'
    :param h: forcasting horizon used in calculating errors (e1, e2)
    :param power: power used in the loss function (usually 1 or 2)
    :return: named tuple containing DM statistic and p-value
    Examples
    --------
    >>> e1 = np.random.rand(100)
    >>> e2 = np.random.rand(100)
    >>> dm_test(e1, e2, h=1, power=2, alternative='two_sided')
    dm_test_result(dm_stat=1.023019805162665, p_value=0.30879181140619405)
    """
    alternatives = ['two_sided', 'less', 'greater']
    if alternative not in alternatives:
        raise ValueError(f"alternative must be one of {alternatives}")

    e1 = check_array(e1, ensure_2d=False)
    e2 = check_array(e2, ensure_2d=False)

    d = np.abs(e1) ** power - np.abs(e2) ** power
    n = d.shape[0]
    d_cov = sm.tsa.acovf(d, fft=True, nlag=h - 1)
    d_var = (d_cov[0] + 2 * d_cov[1:].sum()) / n

    if d_var > 0:
        dm_stat = np.mean(d) / np.sqrt(d_var)
    elif h == 1:
        raise ValueError('Variance of DM statistic is zero')
    else:
        warnings.warn('Variance is negative, using horizon h=1', RuntimeWarning)
        return dm_test(e1, e2, alternative=alternative, h=1, power=power)

    # The corrected statistic suggested by HLN
    k = ((n + 1 - 2 * h + h / n * (h - 1)) / n) ** 0.5
    dm_stat *= k

    if alternative == 'two_sided':
        p_value = 2 * t.cdf(-abs(dm_stat), df=n - 1)
    else:
        p_value = t.cdf(dm_stat, df=n - 1)
        if alternative == 'greater':
            p_value = 1 - p_value

    dm_test_result = collections.namedtuple('dm_test_result', ['dm_stat', 'p_value'])
    return dm_test_result(dm_stat=dm_stat, p_value=p_value)

In [6]:
# Import dataframes

OLS = pd.read_csv("/data/workspace_files/ols.csv")
ENet = pd.read_csv("/data/workspace_files/enet.csv")
PCR = pd.read_csv("/data/workspace_files/pcr.csv")
PLS = pd.read_csv("/data/workspace_files/pls.csv")
RF = pd.read_csv("/data/workspace_files/rf.csv")
GBRT = pd.read_csv("/data/workspace_files/gbrt.csv")

dm_dfs = [OLS, ENet,PLS, PCR, RF, GBRT]

for df in dm_dfs:
    df.sort_values(by = ['MonthYear', 'id'], ascending = True)
  

OLS = OLS.set_index(['MonthYear','id'])
ENet = ENet.set_index(['MonthYear','id'])
PLS = PLS.set_index(['MonthYear','id'])
PCR = PCR.set_index(['MonthYear','id'])
RF = RF.set_index(['MonthYear','id'])
GBRT = GBRT.set_index(['MonthYear','id'])

# #Save inly columns containing excess returns predictions (yhat) and the true excess returns in t+1 (ytrue)
OLS=OLS[["yhat"]]
PLS=PLS[["yhat"]]
ENet=ENet[["yhat"]]
PCR=PCR[["yhat"]]
RF=RF[["yhat"]]
GBRT=GBRT[["yhat"]]

# #rename colums according to the respective model
OLS = OLS.rename(columns={'yhat': 'OLS'})
PLS = PLS.rename(columns={'yhat': 'PLS'})
ENet = ENet.rename(columns={'yhat': 'ENet'})
PCR = PCR.rename(columns={'yhat': 'PCR'})
RF = RF.rename(columns={'yhat': 'RF'})  
GBRT = GBRT.rename(columns={'yhat': 'GBRT'})  


# #prepare for merging
dm_dfs = [OLS, ENet,PLS, PCR, RF, GBRT, df]
for df in dm_dfs:
    df.reset_index(inplace=True)

for df in dm_dfs:
    df["Index"] = df["MonthYear"].astype(str) + df["id"].astype(str)
    
for df in dm_dfs:    
    df.drop(["MonthYear"], axis = 1, inplace = True)
    df.drop(["id"], axis = 1, inplace = True)

#merge dataframes 
df  = reduce(lambda left,right: pd.merge(left,right, on = "Index"), dm_dfs)  

# #Restore multiindex
df["MonthYear"]=df["Index"].str[:7]
df["id"]=df["Index"].str[7:]
df.drop(["Index"], axis = 1, inplace = True)

df

,OLS,ENet,PLS,PCR,RF,GBRT,index,yhat,y_true,date,MonthYear,id
0,0.009476,0.013291,0.013207,0.014044,0.030521,0.013047,0,0.013047,-0.041823,1968-01-31,1968-01,10006
1,0.005314,0.017077,0.013771,0.017937,0.052668,0.016336,1,0.016336,0.087460,1968-01-31,1968-01,10014
2,0.015058,0.019243,0.018217,0.020058,0.030521,0.011766,2,0.011766,0.034743,1968-01-31,1968-01,10030
3,0.024676,0.028924,0.024757,0.029822,0.060628,0.027811,3,0.027811,0.017201,1968-01-31,1968-01,10057
4,0.012692,0.014072,0.013575,0.014823,-0.029947,-0.018461,4,-0.018461,-0.011992,1968-01-31,1968-01,10065
...,...,...,...,...,...,...,...,...,...,...,...,...
2666930,-0.004898,-0.000494,-0.000529,-0.033741,-0.017218,-0.047093,2666930,-0.047093,0.421368,2020-11-30,2020-11,93423
2666931,0.008577,0.009199,0.009349,-0.023944,-0.005471,-0.029541,2666931,-0.029541,0.224361,2020-11-30,2020-11,93426
2666932,0.013230,0.012119,0.012293,-0.021002,-0.004383,-0.025179,2666932,-0.025179,0.138120,2020-11-30,2020-11,93427
2666933,-0.017595,-0.003883,-0.004156,-0.037189,-0.018994,-0.042082,2666933,-0.042082,0.144736,2020-11-30,2020-11,93434


In [7]:
#################################
# DM test
#################################


models = ["OLS","ENet","PLS","PCR","RF","GBRT"]
column_names = ["OLS","ENet","PLS","PCR","RF","GBRT"]
row_names = ["OLS","ENet","PLS","PCR","RF","GBRT"]



DM_table = pd.DataFrame(columns = column_names, index = row_names)
for i1 in range(len(models)): # OLS, ENet, PLS, PCR RF, GBRT
    for i2 in range(len(models)): # OLS, ENet, PLS, PCR, GBRT
        y_true = df.y_true
        models_method1 = models[i1]
        models_method2 = models[i2]       
        y_hat_1 = df[models_method1]
        y_hat_2 = df[models_method2]
        df['e_hat_1'] = y_true - y_hat_1
        df['e_hat_2'] = y_true - y_hat_2
        if df['e_hat_1'].tolist() == df['e_hat_2'].tolist():
            DM_table.iloc[i1,i2] = 0
        else:
            temp = dm_test(df['e_hat_1'], df['e_hat_2'], alternative='two_sided', h=1, power=2)
            DM_stat = temp[0]
            if temp[1]<=0.01:
                DM_stat = (temp[0].astype(str) + "*")
            if 0.01 < temp[1] <= 0.05:
                DM_stat = (temp[0].astype(str) + "**")
            if  0.05 < temp[1] <= 0.10:
                DM_stat = (temp[0].astype(str) + "***")
            DM_table.iloc[i1,i2] = DM_stat

This table reports pairwise Diebold-Mariano test statistics for the pairwise comparison of the predictive performance of each machine learning models. Positive statistic indicates that the column model outperforms the row model. The test statistic is calculated based on the actual and predicted returns on a 10-year sample (January 2010-December 2019). ,*,*** represent statistical signicance at 1%, 5%, and 10% levels

In [8]:
DM_table

,OLS,ENet,PLS,PCR,RF,GBRT
OLS,0,58.83304127645665*,57.51715090351947*,39.907611154210414*,-81.04278680869454*,-188.35523882905204*
ENet,-58.83304127645665*,0,-10.558661709463086*,-52.889493914806124*,-122.97161050084007*,-199.3884990548153*
PLS,-57.51715090351947*,10.558661709463086*,0,-42.50475188399099*,-120.634259448303*,-199.17586227901506*
PCR,-39.907611154210414*,52.889493914806124*,42.50475188399099*,0,-121.40291094954783*,-199.01248785757744*
RF,81.04278680869454*,122.97161050084007*,120.634259448303*,121.40291094954783*,0,-122.33462662761286*
GBRT,188.35523882905204*,199.3884990548153*,199.17586227901506*,199.01248785757744*,122.33462662761286*,0
